# Messages Metrics

Let's dive into your message history! This notebook only works on Macs running macOS 12.6 or greater. For best results, make sure you're using [Messages in iCloud](https://support.apple.com/en-us/HT208532).

**Your messages will not leave this device.**

In [118]:
import os
import sqlite3

chat_db = os.environ.get('HOME') + '/Library/Messages/chat.db'
connection = sqlite3.connect(chat_db)
cursor = connection.cursor()
print("Opened chat database %s" % chat_db)

Opened chat database /Users/jungaretti/Library/Messages/chat.db


In [119]:
import pandas

messages_table = pandas.read_sql_query('''select *, datetime(date/1000000000 + strftime("%s", "2001-01-01") ,"unixepoch","localtime")  as date_utc from message''', connection)
handles_table = pandas.read_sql_query('select * from handle', connection)
print("Read {:,} messages".format(messages_table.shape[0]))

Read 699,127 messages


In [120]:
messages_table['timestamp'] = messages_table['date_utc'].apply(lambda x: pandas.Timestamp(x))
messages_table['date'] = messages_table['timestamp'].apply(lambda x: x.date())
messages_table['year'] = messages_table['timestamp'].apply(lambda x: int(x.year))
messages_table['month'] = messages_table['timestamp'].apply(lambda x: int(x.month))

handles_table.rename(columns={'id': 'handle', 'ROWID': 'handle_id'}, inplace=True)
messages = pandas.merge(messages_table, handles_table, on='handle_id', how='left')
print("Processed {:,} messages".format(messages.shape[0]))

Processed 699,127 messages


## Frequently Asked Questions

Each row of `messages` includes information about a message's `text`, `timestamp`, `phone_number`, and much more.

In [121]:
import matplotlib.pyplot as plt

### Who are my top contacts?

In [ ]:
top_handles = messages.groupby('handle').size().nlargest(12)
top_handles.plot(kind="bar")

plt.title("Who are my top contacts?")
plt.xlabel('Phone Number or Email')
plt.ylabel('Messages')
plt.show()